In [68]:
import os
import json
from html import escape
from typing import List, Dict



In [69]:

with open("data/institutions.json", "r", encoding="utf-8") as f_json:
    institutions_loaded = json.load(f_json)

with open("data/researchers.json", "r", encoding="utf-8") as f_json:
    researchers_loaded = json.load(f_json)

In [70]:

def generate_researchers_html(institution_name: str, output_file: str, researchers: List[Dict]):
    """
    Generate an HTML page listing researchers.

    :param institution_name: Name of the institution (string)
    :param output_file: Output HTML file name (string)
    :param researchers: List of researcher dictionaries
    """
    # Sort researchers by surname (case-insensitive, fallback to empty string)
    researchers = sorted(researchers, key=lambda r: r.get('surname', '').lower())

    html_content = [
        "<!DOCTYPE html>",
        "<html lang='en'>",
        "<head>",
        f"<meta charset='UTF-8'>",
        f"<title>{escape(institution_name)} - Researchers</title>",
        "<style>",
        "body { font-family: Arial, sans-serif; margin: 20px; background: #f9f9f9; }",
        "h1 { color: #2c3e50; }",
        ".researcher { border: 1px solid #ddd; padding: 15px; background: #fff; margin: 15px 0;",
        "          border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }",
        ".researcher h2 { margin-top: 0; color: #34495e; }",
        #"p { margin: 5px 0; font-size: 14px; color: #444; }",
        "a { color: #0073e6; text-decoration: none; }",
        "a:hover { text-decoration: underline; }",
        "</style>",
        "</head>",
        "<body>",
        f"<h1>Researchers at {escape(institution_name)}</h1>",
    ]

    for i, r in enumerate(researchers):
        html_content.append("<div class='researcher'>")

        # Full Name
        surname = escape(r.get('surname', ''))
        name = escape(r.get('name', ''))
        if surname and name:
            full_name = f"{surname}, {name}"
        else:
            full_name = surname or name
        if full_name:
            html_content.append(f"<h2>{i + 1}. {full_name}</h2>")

        # Position
        if r.get("position"):
            html_content.append(f"<p><strong>Position:</strong> {escape(r['position'])}</p>")

        # Department
        if r.get("department"):
            html_content.append(f"<p><strong>Department:</strong> {escape(r['department'])}</p>")

        # Area/Section
        if r.get("area_or_section"):
            html_content.append(f"<p><strong>Area/Section:</strong> {escape(r['area_or_section'])}</p>")

        # Research Interests
        if r.get("research_interests"):
            html_content.append(f"<p><strong>Research Interests:</strong> {escape(r['research_interests'])}</p>")

        # Email
        if r.get("email"):
            html_content.append(f"<p><strong>Email:</strong> <a href='mailto:{escape(r['email'])}'>{escape(r['email'])}</a></p>")

        # Personal Webpage
        if r.get("link_to_webpage"):
            html_content.append(f"<p><a href='{escape(r['link_to_webpage'])}' target='_blank'>Personal Webpage</a></p>")

        # Institution
        if r.get("institution_name"):
            institution_html = escape(r["institution_name"])
            if r.get("institution_url"):
                institution_html = f"<a href='{escape(r['institution_url'])}' target='_blank'>{institution_html}</a>"
            html_content.append(f"<p><strong>Institution:</strong> {institution_html}</p>")

        html_content.append("</div>")

    html_content.append("</body></html>")

    # Write to file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(html_content))

In [71]:


def generate_main_page(institutions, researchers, output_file="index.html", output_dir="pages"):
    """
    Generate a styled HTML main page listing institutions with department and researcher icon.

    :param institutions: List of institution dictionaries
    :param researchers: List of researcher dictionaries
    :param output_file: Output HTML file name
    """

    # Sort alphabetically by institution name
    institutions.sort(key=lambda x: x["name"].lower())

    # HTML Header
    html = [
        "<!DOCTYPE html>",
        "<html lang='en'>",
        "<head>",
        "    <meta charset='UTF-8'>",
        "    <title>Institutions</title>",
        "    <style>",
        "        body { font-family: Arial, sans-serif; margin: 20px; background: #f9f9f9; }",
        "        h1 { color: #2c3e50; }",
        "        .institution { border: 1px solid #ddd; background: #fff; padding: 15px; margin: 15px 0; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }",
        "        .institution h2 { margin-top: 0; color: #34495e; }",
        #"        p { margin: 5px 0; font-size: 14px; color: #444; }",
        "        a { color: #0073e6; text-decoration: none; }",
        "        a:hover { text-decoration: underline; }",
        "        .researcher-icon { width: 16px; height: 16px; vertical-align: middle; }",
        "        .department-icon { width: 16px; height: 16px; vertical-align: baseline; }",
        "    </style>",
        "</head>",
        "<body>",
        "    <h1>Institutions</h1>"
    ]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Generate institution cards
    for i, inst in enumerate(institutions):
        # Generate individual researcher pages
        institution_name = inst["name"]
        researchers_page = inst['researcher_page']
        researcher_output_file = output_dir + '/' + researchers_page
        # Filter researchers for the current institution
        institution_researchers = [r for r in researchers if r.get("institution_name") == institution_name]
        # Generate HTML page
        generate_researchers_html(institution_name, researcher_output_file, institution_researchers)
        print(f"Generated {researcher_output_file} with {len(institution_researchers)} researchers.")
        
        html.append("<div class='institution'>")
        # Institution name as h2
        html.append(f"<h2>{i+1}. {escape(inst['name'])}</h2>")
        # Department
        if inst.get("department_name") and inst.get("department_url"):
            html.append(f"<p><strong></strong> <a href='{escape(inst['department_url'])}' target='_blank'><img src='pages/icon_department.png' alt='Department' class='department-icon'> {escape(inst['department_name'])}</a></p>")
        # Researchers link with icon
        if inst.get("researcher_page"):
            html.append(f"<p><a href='pages/{escape(inst['researcher_page'])}' title='Researchers'><img src='pages/icon_list.png' alt='Researchers' class='researcher-icon'> Researchers ({len(institution_researchers)})</a></p>")
        html.append("</div>")

    # HTML Footer
    html.extend([
        "</body>",
        "</html>"
    ])

    # Write to file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(html))

    print(f"Main page generated: {output_file}")

In [72]:
generate_main_page(institutions_loaded, researchers_loaded, "index.html")


Generated pages/researchers_acu.html with 9 researchers.
Generated pages/researchers_baylor.html with 51 researchers.
Generated pages/researchers_etamu.html with 11 researchers.
Generated pages/researchers_hsutx.html with 4 researchers.
Generated pages/researchers_msutexas.html with 7 researchers.
Generated pages/researchers_pvamu.html with 20 researchers.
Generated pages/researchers_rice.html with 30 researchers.
Generated pages/researchers_sam_houston.html with 12 researchers.
Generated pages/researchers_smu.html with 22 researchers.
Generated pages/researchers_stmarytx.html with 8 researchers.
Generated pages/researchers_tarleton.html with 12 researchers.
Generated pages/researchers_tamiu.html with 20 researchers.
Generated pages/researchers_texas_am.html with 68 researchers.
Generated pages/researchers_tamuc.html with 13 researchers.
Generated pages/researchers_tamuk.html with 19 researchers.
Generated pages/researchers_tamut.html with 4 researchers.
Generated pages/researchers_tcu